In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 2.7.15+ (default, Nov 27 2018, 23:36:35) 
[GCC 7.3.0]
NLTK: 3.2.5
Scikit-learn: 0.20.3
Pandas: 0.24.2
Numpy: 1.14.1


In [64]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('data/dataset_tratado.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null bool
1    793 non-null object
dtypes: bool(1), object(1)
memory usage: 7.0+ KB
None
       0                                                  1
0  False  inexecuca contrat sanca afast reexam prov clau...
1  False  acident aer decretol n dol eventual respons at...
2  False  intimaca art cpc irregular nao comprov recurs ...
3  False  honorari execuca legitim part superior tribuna...
4  False  servidor public artig lei n artig lei n revoga...


In [70]:
# verificação das classes dos acórdãos

binary = df[0]
print(binary.value_counts())

False    525
True     268
Name: 0, dtype: int64


In [75]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes
encoder = LabelEncoder()
binary = encoder.fit_transform(binary)

print(binary[:10])
len(binary)

[0 0 0 0 0 0 1 1 0 0]


793

In [67]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    inexecuca contrat sanca afast reexam prov clau...
1    acident aer decretol n dol eventual respons at...
2    intimaca art cpc irregular nao comprov recurs ...
3    honorari execuca legitim part superior tribuna...
4    servidor public artig lei n artig lei n revoga...
5    iss arrend mercantil incidenc mat constitucion...
6    processual penal art codig process penal conhe...
7    recurs especial art incis art ambos codig pena...
8    praz prescricional lei complement n artig lei ...
9    licitaca aca anulator indefer tutel antecip ag...
Name: 1, dtype: object


In [72]:
# uso de word2vec para a extração de features

from gensim.models import Word2Vec

ementas_list = [word for word in ementas.iteritems()]
ementas_list_list = []
for doc in ementas_list:
    ementas_list_list.append(doc[1].split())

X = Word2Vec(ementas_list_list, min_count=2)
X.wv.init_sims() # para inicializar model.vw.syn0norm, necessário para o cálculo da média da
print(X)

Word2Vec(vocab=3410, size=100, alpha=0.025)


In [81]:
# funções para calcular a média vetorial das palavras de cada ementa

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [78]:
# função para tokenizar as palavras com frequência maior que 2

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [82]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


train, test = train_test_split(df, test_size=0.2, random_state=0)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values

X_train = word_averaging_list(X.wv,train_tokenized)
X_test = word_averaging_list(X.wv,test_tokenized)

y_train = train[0]
y_test = test[0]

print(X_train)
print(X_test)

[[ 0.14534767  0.01982272  0.0448398  ... -0.07477809  0.02976305
  -0.04991043]
 [ 0.12520508  0.02962115  0.05239616 ... -0.06825581  0.03478844
  -0.02642938]
 [ 0.1306565   0.04340264  0.04938867 ... -0.07956544  0.02947289
  -0.02057191]
 ...
 [ 0.13771681  0.02761975  0.0499327  ... -0.07228079  0.030251
  -0.0448534 ]
 [ 0.1387119   0.04569263  0.04449716 ... -0.07536625  0.03929668
  -0.03407298]
 [ 0.13451058  0.02732372  0.05012171 ... -0.07068528  0.03359158
  -0.03754881]]
[[ 0.13530824  0.04177387  0.04626328 ... -0.07259557  0.03818455
  -0.03387338]
 [ 0.17629535 -0.0101957   0.03212216 ... -0.08912148  0.02519407
  -0.06986885]
 [ 0.1120663   0.04584334  0.05504971 ... -0.0656649   0.03239399
  -0.01685729]
 ...
 [ 0.12619235  0.03421398  0.04937165 ... -0.07320057  0.03131828
  -0.02328027]
 [ 0.12410226  0.03746981  0.05848242 ... -0.05842689  0.04324841
  -0.03667566]
 [ 0.16727343 -0.00660453  0.03137209 ... -0.09150209  0.02583397
  -0.04554113]]


In [83]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs'),
    SGDClassifier(max_iter = 100),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 61.6352201258
Decision Tree Accuracy: 70.4402515723
Random Forest Accuracy: 71.0691823899
Logistic Regression Accuracy: 62.893081761
SGD Classifier Accuracy: 62.893081761
Naive Bayes Accuracy: 62.893081761


In [84]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs'),
    SGDClassifier(max_iter = 100),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 66.6666666667


In [86]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

[[93  7]
 [46 13]]
              precision    recall  f1-score   support

       False       0.67      0.93      0.78       100
        True       0.65      0.22      0.33        59

   micro avg       0.67      0.67      0.67       159
   macro avg       0.66      0.58      0.55       159
weighted avg       0.66      0.67      0.61       159

66.66666666666666
